# Placeholders
在前面我们介绍计算图的时候, 我们注意到构建图的时候可以脱离具体值进行定义它的整体结构, 在运行的时候可以根据需要带入具体的数值. 可是前面我们定义的常量tensor以及变量tensor都需要一个初始值.

因此, 为了更加契合图的构建过程, tensorflow引入了一个占位符.(placeholder)的概念. 字如其面, 它只是占着构建图的一个位置,没有具体数值, 但必须要有具体的类型以及形状. 

In [1]:
import tensorflow as tf

In [2]:
##定义一个占位符
##`tf.placeholder(dtype, shape=None, name=None)`

# 定义一个`float32`型的占位符,它是一个⻓为3的向量
a = tf.placeholder(tf.float32, shape=[3])

# 定义一个`bool`型的占位符, 它是一个`1x2`的矩阵
b = tf.placeholder(tf.bool, shape=[1,2])

如果我们还像之前那样企图用a.eval()的话, 就会体会到placeholder的特别之处, 因为它真的没有任何值! 实际上,在执行图的过程中, 我们必须要用字典的方式给placeholder喂入具体值, 这个过程称为feed.

表现在程序中, 就是我们在获取一个占位符的值的时候, 需要给run()增加一个feed_dict的参数, 这个参数是一个dict, 它的key是占位符的变量名, 它的val是需要喂入的具体值. 当然, 这个字典可以有很多个key, 也就是说可以一次喂入很多个占位符

In [3]:
with tf.Session() as sess :
    print(sess.run(a, feed_dict={a:[1,2,3]}))
    print(sess.run(b, feed_dict={b:[[True,False]]}))

[1. 2. 3.]
[[ True False]]


In [4]:
x = tf.placeholder(tf.float32, shape=[4])
y = tf.square(x)
with tf.Session() as sess:
    print(sess.run(y,feed_dict={x:[1,2,4,8]}))
    print(y.eval(feed_dict={x:[1,2,4,8]}))

[ 1.  4. 16. 64.]
[ 1.  4. 16. 64.]


# tf.Graph
在执行图之前, 需要将整个计算图都构建完成, 那么我们如何才能获得这个完整的图呢?

In [5]:
var_a = tf.Variable(5,dtype=tf.float32, name='var_a')
var_b = tf.Variable(2,dtype=tf.float32, name='var_b')
var_c = tf.add(var_a,var_b, name='op_add')

init = tf.global_variables_initializer()
g = tf.get_default_graph()
print(g)

得到这个图之后, 我们当然可以往前回溯我们创建过的图的每一个节点, 通过g.get_operations()就可以查到所有的节点

In [6]:
for op in g.get_operations():
    print(op.name)

Placeholder
Placeholder_1
Placeholder_2
Square
var_a/initial_value
var_a
var_a/Assign
var_a/read
var_b/initial_value
var_b
var_b/Assign
var_b/read
op_add
init


我们还可以通过g.get_tensor_by_name()获得对应name的tensor<br>注意：0表示第0个gpu,初始化后的tensor与原tensor命名不同

In [7]:
with tf.Session() as sess:
    what_is_this = g.get_tensor_by_name('var_a/initial_value:0')
    print(what_is_this.eval())

5.0


# 生成新图
上面介绍了tensorflow运行程序时的默认图, 那么我们可以构造有别于默认图的新图吗? 当然可以的, 通过g1=tf.Graph()就可以办到. 但这并没有结束, 我们需要将这个图设置为我们构造tensor以及op的默认图, 让程序能够理解我们将要定义的tensor到底在哪个图里定义, 因此我们还需要通过python的with语句来确立图的作用范围.然后我们可以通过变量的.graph属性来确定它在哪个图中

In [9]:
##定义一个新图, 注意它和之前的g有什么不同
g1 = tf.Graph()
print('g1:',g1)

g1: <tensorflow.python.framework.ops.Graph object at 0x0000023DCD0F5940>


In [10]:
print('default_graph',tf.get_default_graph())
g1.as_default()
print('default_graph',tf.get_default_graph())

default_graph <tensorflow.python.framework.ops.Graph object at 0x0000023DB78A57B8>
default_graph <tensorflow.python.framework.ops.Graph object at 0x0000023DB78A57B8>


In [12]:
a1 = tf.constant(32, name='a1')

with g1.as_default():
    a2 = tf.constant(32, name='a2')
    
print('a.graph: ', a.graph)
print('a1.graph: ', a1.graph)
print('a2.graph: ', a2.graph)

a.graph:  <tensorflow.python.framework.ops.Graph object at 0x0000023DB78A57B8>
a1.graph:  <tensorflow.python.framework.ops.Graph object at 0x0000023DB78A57B8>
a2.graph:  <tensorflow.python.framework.ops.Graph object at 0x0000023DCD0F5940>


# 图的可视化
现在我们要迫不及待的给大家介绍tensorflow的一款神器tensorboard!它在我们安装tensorflow的过程中就已经被自动安装了, 非常方便.在它的帮助下, 我们可以将刚才我们构造的图可视化, 让它变得更加清晰直观

In [13]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('graph',sess.graph)

这个时候在当前目录中你就会多一个events.*的文件, 这个就是需要的文件.然后我们在当前目录打开终端(一般来说是右键, 点击Open in New Terminal或者是在终端中打开),键入以下命令:<br>`$ tensorboard --logdir=` <br> 然后就会有一个这样的输出 <br> TensorBoard 0.1.8 at http://USERNAME:6006 (Press CTRL+C to quit)<br>然后我们打开浏览器, 输入at后面的链接 http://USERNAME:6006 进
